In [1]:
import keras.losses
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image

#from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.applications.xception import preprocess_input

from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import img_to_array
from keras import layers, Model
from keras.regularizers import l2
import numpy as np
import pandas as pd
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential, Input, load_model
from sklearn.utils import class_weight
import datetime
import zipfile

In [2]:
train_triplets = pd.read_csv("train_triplets.txt", sep=' ', header=None, dtype=str)
test_triplets = pd.read_csv("test_triplets.txt", sep=' ', header=None, dtype=str)

""" with zipfile.ZipFile("food.zip","r") as zip_ref:
    zip_ref.extractall("food_porn") """

' with zipfile.ZipFile("food.zip","r") as zip_ref:\n    zip_ref.extractall("food_porn") '

In [7]:

with zipp.zipfile("food.zip","r") as zip_ref:
    zip_ref.extractall("food_porn")

TypeError: 'module' object is not callable

In [3]:
def append_ext(index):
    return index + '.jpg'

In [4]:
ONE_SET_SIZE = 59515
ABC = ['A', 'B', 'C']

train_triplets_jpg = train_triplets.apply(append_ext)
#data_y1 = pd.DataFrame(np.ones((ONE_SET_SIZE)), dtype='str', columns=['label'])
#train_triplets_jpg = pd.concat([train_triplets_jpg, data_y1], axis=1)
train_triplets_jpg = train_triplets_jpg.rename(columns={0:'A', 1:'B', 2:'C'}, inplace=False)

test_triplets_jpg = test_triplets.apply(append_ext)
test_triplets_jpg = test_triplets_jpg.rename(columns={0:'A', 1:'B', 2:'C'}, inplace=False)

In [5]:
%%time


# There are 4 different permutations of ABC that we can know the answer of. (CAB and CBA we don't know)
ONE_SET_SIZE = 59515

#create the base dataset with the 2 sets of labels
data_y1 = pd.DataFrame(np.ones((ONE_SET_SIZE * 2,1)), dtype='str', columns=['label'])
data_y0 = pd.DataFrame(np.zeros((ONE_SET_SIZE * 2,1)), dtype='str', columns=['label'])

#create the inverse dataset and a set where B is connected to A (and inv)

# One needs 3 different assignments since the arrays would be linked otherwise.
ACB0 = train_triplets_jpg[['A', 'C', 'B']].copy()
BAC1 = train_triplets_jpg[['B', 'A', 'C']].copy()
BCA0 = train_triplets_jpg[['B', 'C', 'A']].copy()

ACB0 = ACB0.rename(columns={'C':'B', 'B':'C'}, inplace=False)
BAC1 = BAC1.rename(columns={'B':'A', 'A':'B'}, inplace=False)
BCA0 = BCA0.rename(columns={'B':'A', 'C':'B', 'A':'C'}, inplace=False)

id_fin = pd.concat((train_triplets_jpg, BAC1, ACB0, BCA0),axis=0, ignore_index=True)
labels = pd.concat((data_y1,data_y0),axis=0, ignore_index=True)

data_nn_x = pd.concat([id_fin,labels], join='outer', axis=1)

""" # This only shuffels the set which might be unneccessary since ImageGen also shuffles. Still for redundanacy. Costs only 70ms on my machiene.
data_nn_x = data_nn_x.sample(frac=1).reset_index(drop=True) """ 

Wall time: 99 ms


' # This only shuffels the set which might be unneccessary since ImageGen also shuffles. Still for redundanacy. Costs only 70ms on my machiene.\ndata_nn_x = data_nn_x.sample(frac=1).reset_index(drop=True) '

In [6]:
#print(data_nn_x.loc[3,'label'].dtype)

In [6]:
batch_size = 48
pixels = 144

In [7]:
datagen=image.ImageDataGenerator(
    rescale=1./255.,
    validation_split=0.2,
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

test_datagen=image.ImageDataGenerator(
    rescale=1./255.,
    preprocessing_function=preprocess_input,
    horizontal_flip=True
    )

In [8]:
def generate_generator_multiple(generator, triplet_file, batch_size, pixels, subset):

    if subset == 'training':
        shuffle = True
        y_col = 'label'
        class_mode = 'binary'
    elif subset == 'validation':
        shuffle = False
        y_col = 'label'
        class_mode = 'binary'
    elif subset == None:
        shuffle = False
        y_col = None
        class_mode = 'input'
    else: raise Exception('Subset needs to be either training, validation or None.')

    generator_A = datagen.flow_from_dataframe(
        triplet_file,
        directory='./food_porn/food',
        x_col= 'A',
        y_col=y_col,
        subset=subset ,
        shuffle=shuffle,
        class_mode=class_mode,
        target_size=(pixels, pixels),
        batch_size=batch_size,
        seed=69,
        )

    generator_B = datagen.flow_from_dataframe(
        triplet_file,
        directory='./food_porn/food',
        x_col= 'B',
        y_col=y_col,
        subset=subset,
        shuffle=shuffle,
        class_mode=class_mode,
        target_size=(pixels, pixels),
        batch_size=batch_size,
        seed=69,
        )


    generator_C = datagen.flow_from_dataframe(
        triplet_file,
        directory='./food_porn/food',
        x_col= 'C',
        y_col=y_col,
        subset=subset,
        shuffle=shuffle,
        class_mode=class_mode,
        target_size=(pixels, pixels),
        batch_size=batch_size,
        seed=69,
    )

    while True:
            X1i = generator_A.next()
            X2i = generator_B.next()
            X3i = generator_C.next()

            yield [X1i[0], X2i[0], X3i[0]], X3i[1]  #Yield all images and their mutual label

In [9]:
input_generator = generate_generator_multiple(generator=datagen,
                                            triplet_file=data_nn_x,
                                            batch_size=batch_size,
                                            pixels=pixels,
                                            subset='training')       
     
validation_generator = generate_generator_multiple(generator=datagen,
                                            triplet_file=data_nn_x,
                                            batch_size=batch_size,
                                            pixels=pixels,
                                            subset='validation') 

test_generator = generate_generator_multiple(generator=test_datagen,
                                            triplet_file=test_triplets_jpg,
                                            batch_size=batch_size,
                                            pixels=pixels,
                                            subset=None)

In [46]:
i = 0

for batch in datagen.flow_from_dataframe(
        data_nn_x,
        directory='./food_porn/food',
        x_col= 'A',
        y_col='label',
        subset='training' ,
        shuffle=False,
        class_mode='binary',
        target_size=(144, 144),
        batch_size=1,
        seed=69,
        save_to_dir='./flow/A'
        ):

    i+=1
    if i> 5:
        break


Found 190448 validated image filenames belonging to 2 classes.


In [47]:
i= 0

for batch in datagen.flow_from_dataframe(
        data_nn_x,
        directory='./food_porn/food',
        x_col= 'B',
        y_col='label',
        subset='training' ,
        shuffle=False,
        class_mode='binary',
        target_size=(144, 144),
        batch_size=1,
        seed=69,
        save_to_dir='./flow/B'
        ):

    i+=1
    if i> 5:
        break


Found 190448 validated image filenames belonging to 2 classes.


In [29]:
cnn = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')

""" Comment this when uploading to euler. """

' Comment this when uploading to euler. '

In [10]:
cnn.save('ResNetV2.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
del cnn

In [5]:
import zipp

In [4]:
#cnn.summary()

NameError: name 'cnn' is not defined

In [13]:
# Multiple inputs
in1 = layers.Input(shape=(pixels,pixels,3))
in2 = layers.Input(shape=(pixels,pixels,3))
in3 = layers.Input(shape=(pixels,pixels,3))

# CNN output
#cnn = applications.xception.Xception(include_top=False)
cnn = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')
#cnn = load_model('ResNetV2')

#cnn.summary()

for layer in cnn.layers[:-4]:
    layer.trainable = False

out1 = cnn(in1)
out2 = cnn(in2)
out3 = cnn(in3)

# Concatenating the final output
out = layers.Concatenate(axis=-1)([out1, out2, out3])
out = Dropout(0.5)(out)
combined = Dense(units = 1024, activation='relu', kernel_regularizer=l2(0.003))(out)
combined = Dropout(0.5)(combined)
combined = Dense(units = 1, activation='sigmoid')(combined)

# Creating the model
model = Model(inputs=[in1, in2, in3], outputs=combined)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 144, 144, 3) 0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 144, 144, 3) 0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 144, 144, 3) 0                                            
__________________________________________________________________________________________________
inception_resnet_v2 (Functional (None, 1536)         54336736    input_5[0][0]                    
                                                                 input_6[0][0]              

In [11]:
val_step = np.floor(59515*4*0.2/batch_size)
ep_step = np.floor(59515*4*0.8/batch_size)

In [12]:
model.compile(loss = keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])

#fit the network to (for now the un-sparse matrix)
model.fit(input_generator, validation_data=validation_generator, epochs= 64,
          batch_size=batch_size, verbose=1, steps_per_epoch = 200,
          validation_steps= 10)

Found 190448 validated image filenames belonging to 2 classes.
Found 190448 validated image filenames belonging to 2 classes.
Found 190448 validated image filenames belonging to 2 classes.
Epoch 1/64
200/200 [==============================] - ETA: 0s - loss: 0.7122 - accuracy: 0.5822Found 47612 validated image filenames belonging to 2 classes.
Found 47612 validated image filenames belonging to 2 classes.
Found 47612 validated image filenames belonging to 2 classes.
200/200 [==============================] - 1158s 6s/step - loss: 0.7121 - accuracy: 0.5823 - val_loss: 0.9488 - val_accuracy: 0.0000e+00
Epoch 2/64
  4/200 [..............................] - ETA: 18:05 - loss: 0.6903 - accuracy: 0.6016

KeyboardInterrupt: 

In [ ]:
predict_test_nn = model.predict(test_generator, batch_size=batch_size)

In [ ]:
export = np.array(predict_test_nn>0.5).astype('int')
np.savetxt("submission_generator.txt", np.round(export,decimals=0), fmt='%i')